In [1]:
from typing import List, Set
import random
import matplotlib.pyplot as plt
import networkx as nx
import scipy
from tqdm import tqdm
import time
import itertools

from networkx.algorithms.approximation.traveling_salesman import christofides
from networkx.utils import pairwise

In [2]:
class Tableau:
    idx: int
    style: str           # Landscape or Portrait
    nb_text: int         # Nb texts
    tags: Set[str]       # Set of tags

    def __init__(self, ligne, idx):
        self.idx     = idx
        infos        = ligne.split()
        self.style   = infos[0]
        self.nb_text = int(infos[1])
        self.tags    = set(infos[2:])
    
    def __repr__(self):
        return f"Tableau(style={self.style}, tags={self.tags})"

In [3]:
class FrameGlass:
    idx: str
    tableaux: List[Tableau]
    tags: Set[str]
        
    def __repr__(self):
        return f"FrameGlass(id={self.idx},tags={self.tags})"
    

class PortraitFG(FrameGlass):    
    def __init__(self, P1, P2):
        self.tableaux = [P1, P2]
        self.tags = P1.tags.union(P2.tags)
        self.idx = str(P1.idx) + 'P' + str(P2.idx)


class LandscapeFG(FrameGlass):
    def __init__(self, L):
        self.tableaux = [L]
        self.tags = L.tags
        self.idx = str(L.idx) + 'L' 

In [4]:
def load_tableaux(file):
    tableaux = []
    with open(file) as f:
        line = f.readline()
        line = f.readline()
        idx = 1
        while line:
            tableaux.append(Tableau(line, idx))
            idx += 1
            line = f.readline()
    return tableaux

In [5]:
def satisfaction(fg1, fg2):
    return min(len(fg1.tags - fg2.tags), len(fg1.tags & fg2.tags), len(fg2.tags - fg1.tags))

def total_satisfaction(list_fg):
    return sum([satisfaction(fg1, fg2) for fg1, fg2 in zip(list_fg[:-1], list_fg[1:])])

In [6]:
# Optimisation algorithms : Traveler salesmans problem

def greedy_tsp(G, score="score", source=None, monitor = True):
    
    if source is None:
        source = nx.utils.arbitrary_element(G)

    if G.number_of_nodes() == 2:
        neighbor = next(G.neighbors(source))
        return [source, neighbor, source]

    nodeset = set(G)
    nodeset.remove(source)
    path = [source]
    next_node = source
    if monitor:
        with tqdm(total=len(nodeset)) as pbar:
            while nodeset:
                nbrdict = G[next_node]
                # Récupérer un élément au hasard dans nodeset et le remettre
                random_available = nodeset.pop()
                nodeset.add(random_available)
                candidates = [k for k in nbrdict.keys() if k in nodeset] + [random_available]
                next_node = max(candidates, key=lambda n: nbrdict[n].get(score) if n in nbrdict else 0)
                path.append(next_node)
                nodeset.remove(next_node)
                pbar.update(1)
    else:
        while nodeset:
            nbrdict = G[next_node]
            # Récupérer un élément au hasard dans nodeset et le remettre
            random_available = nodeset.pop()
            nodeset.add(random_available)
            candidates = [k for k in nbrdict.keys() if k in nodeset] + [random_available]
            next_node = max(candidates, key=lambda n: nbrdict[n].get(score) if n in nbrdict else 0)
            path.append(next_node)
            nodeset.remove(next_node)
    return path

In [7]:
def random_fgs(tableaux):
    landscapes = [t for t in tableaux if t.style == 'L']
    portraits  = [t for t in tableaux if t.style == 'P']
    
    random.shuffle(portraits)
    
    portraits_1, portraits_2 = portraits[::2], portraits[1::2]
    
    fgs = [LandscapeFG(t) for t in landscapes] + [PortraitFG(t1, t2) for t1, t2 in zip(portraits_1, portraits_2)]
    
    return fgs

def equitags_fgs(tableaux):
    landscapes = [t for t in tableaux if t.style == 'L']
    portraits  = [t for t in tableaux if t.style == 'P']
    
    portraits.sort(key = lambda t: len(t.tags))
    
    l = len(portraits)
    
    portraits_1, portraits_2 = portraits[:l//2], portraits[-1:(l//2):-1]
    
    fgs = [LandscapeFG(t) for t in landscapes] + [PortraitFG(t1, t2) for t1, t2 in zip(portraits_1, portraits_2)]
    
    return fgs

def store_data(fgs):
    idx_to_tags = {fg.idx:fg.tags for fg in fgs}
    idx_to_fg   = {fg.idx:fg for fg in fgs}
    
    available_tags = set().union(*[fg.tags for fg in fgs])
    
    tag_to_idxs = {t:[] for t in available_tags}
    for idx, tags in idx_to_tags.items():
        for tag in tags:
            tag_to_idxs[tag].append(idx)
    
    return idx_to_tags, idx_to_fg, available_tags, tag_to_idxs

def divide(fgs, n_batchs):
    return [ fgs[k*n_batchs:(k+1)*n_batchs] for k in range(len(fgs)//n_batchs) ]

def create_graph(couples, idx_to_tags, idx_to_fg):    
    G = nx.Graph()
    G.add_nodes_from(idx_to_tags)
    G.add_edges_from(couples, score = 0)
    for idx_start,idx_end in couples:
        G[idx_start][idx_end]['score'] = satisfaction(idx_to_fg[idx_start], idx_to_fg[idx_end])
    return G

def find_all_couples(tag_to_idxs):
    couples = []
    for tag, idxs in tqdm(tag_to_idxs.items()):
        for a,b in itertools.combinations(idxs, 2):            
            couples.append((a,b))
    return couples


In [8]:
def dq_assign(tableaux, size=500):
    # Initialising frameglasses
    fgs = random_fgs(tableaux)

    # Storing the data into dictionnaries
    idx_to_tags, idx_to_fg, available_tags, tag_to_idxs = store_data(fgs)

    # Divide
    li_fgs = divide(fgs, size)

    results = []
    
    for small in tqdm(li_fgs):
        
        indexes = [s.idx for s in small]
        
        couples = list(itertools.combinations(indexes, 2))
        
        G = create_graph(couples, indexes, idx_to_fg)
        
        path = greedy_tsp(G, monitor = False)
        
        small_fgs = [idx_to_fg[i] for i in path]
        
        results += small_fgs

    return results

def general_assign(tableaux):
    
    # Initialising frameglasses
    fgs = random_fgs(tableaux)
    
    # Storing the data into dictionnaries
    idx_to_tags, idx_to_fg, available_tags, tag_to_idxs = store_data(fgs)
            
    # Find all couples
    couples = find_all_couples(tag_to_idxs)
            
    #Create a graph
    G = create_graph(couples, idx_to_tags, idx_to_fg)

    path = greedy_tsp(G)
    
    fgs = [idx_to_fg[idx] for idx in path]
    
    return fgs

def randomizing_assign(tableaux):
    
    fgs = random_fgs(tableaux)
    
    return fgs

In [9]:
def write_results(fgs, fname):
    with open(fname, 'w') as f:
        f.write(str(len(fgs))+'\n')
        for fg in fgs:
            line = fg.idx.replace('L', '').replace('P', ' ')+'\n'
            f.write(line)

In [10]:
# Available : '0_example.txt', '1_binary_landscapes.txt', '10_computable_moments.txt', '11_randomizing_paintings.txt', '110_oily_portraits.txt'
filename = '1_binary_landscapes'

file     = filename + '.txt'
file_out = filename + '_solution.txt'
tableaux = load_tableaux(file)
fgs = general_assign(tableaux)
print('Total satisfaction : ', total_satisfaction(fgs))
write_results(fgs, file_out)

100%|████████████████████████████████████████████████████████████████████████| 79999/79999 [00:00<00:00, 159923.73it/s]


Total satisfaction :  205677


In [11]:
filename = '10_computable_moments' 

file     = filename + '.txt'
file_out = filename + '_solution.txt'
tableaux = load_tableaux(file)
fgs = general_assign(tableaux)
print('Total satisfaction : ', total_satisfaction(fgs))
write_results(fgs, file_out)

100%|█████████████████████████████████████████████████████████████████████████████| 749/749 [00:00<00:00, 22088.17it/s]


Total satisfaction :  1498


In [12]:
filename = '11_randomizing_paintings'

file     = filename + '.txt'
file_out = filename + '_solution.txt'
tableaux = load_tableaux(file)
fgs = dq_assign(tableaux, size = 1000)
print('Total satisfaction : ', total_satisfaction(fgs))
write_results(fgs, file_out)

100%|██████████████████████████████████████████████████████████████████████████████████| 60/60 [02:26<00:00,  2.44s/it]


Total satisfaction :  379547


In [14]:
filename = '110_oily_portraits'

file     = filename + '.txt'
file_out = filename + '_solution.txt'
tableaux = load_tableaux(file, size = 1000)
fgs      = dq_assign(tableaux)
print('Total satisfaction : ', total_satisfaction(fgs))
write_results(fgs, file_out)

100%|██████████████████████████████████████████████████████████████████████████████████| 80/80 [01:16<00:00,  1.05it/s]


Total satisfaction :  304453
